<a href="https://colab.research.google.com/github/ammfat/pyspark-zero-to-hero/blob/colab/22_spark_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Spark SQL")
    .master("local[*]")
    .enableHiveSupport()
    .config("spark.sql.warehouse.dir", "/data/output/spark-warehouse")
    .getOrCreate()
)

spark

In [8]:
!mkdir -p /data/input

!wget https://media.githubusercontent.com/media/ammfat/pyspark-zero-to-hero/refs/heads/colab/datasets/employee_records.csv
!wget https://media.githubusercontent.com/media/ammfat/pyspark-zero-to-hero/refs/heads/colab/datasets/department_data.csv

!mv ./employee_records.csv /data/input/employee_records_skewed.csv
!mv ./department_data.csv /data/input

--2025-08-14 20:17:19--  https://media.githubusercontent.com/media/ammfat/pyspark-zero-to-hero/refs/heads/colab/datasets/employee_records.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97427001 (93M) [text/plain]
Saving to: ‘employee_records.csv’

employee_records.cs 100%[===================>]  92.91M   327MB/s    in 0.3s    

2025-08-14 20:17:19 (327 MB/s) - ‘employee_records.csv’ saved [97427001/97427001]

--2025-08-14 20:17:19--  https://media.githubusercontent.com/media/ammfat/pyspark-zero-to-hero/refs/heads/colab/datasets/department_data.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.

In [10]:
# Read Employee data
_schema = "first_name string, last_name string, job_title string, dob string, email string, phone string, salary double, department_id int"

emp = spark.read.format("csv").schema(_schema).option("header", True).load("/data/input/employee_records_skewed.csv")

In [11]:
# Read DEPT CSV data
_dept_schema = "department_id int, department_name string, description string, city string, state string, country string"

dept = spark.read.format("csv").schema(_dept_schema).option("header", True).load("/data/input/department_data.csv")

In [12]:
# Spark Catalog (Metadata) - in-memory/hive

spark.conf.get("spark.sql.catalogImplementation")

'hive'

In [14]:
# Show databases
db = spark.sql("show databases")
db.show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [15]:
spark.sql("show tables in default").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [16]:
# Register dataframes are temp views

emp.createOrReplaceTempView("emp_view")

dept.createOrReplaceTempView("dept_view")


In [17]:
# Show tables/view in catalog



In [18]:
# View data from table

emp_filtered = spark.sql("""
    select * from emp_view
    where department_id = 1
""")

In [19]:
emp_filtered.show()

+-----------+---------+--------------------+----------+--------------------+--------------------+--------+-------------+
| first_name|last_name|           job_title|       dob|               email|               phone|  salary|department_id|
+-----------+---------+--------------------+----------+--------------------+--------------------+--------+-------------+
|       John|   Monroe|        Retail buyer|1968-06-16|  erik33@example.net|    820-813-0557x624|485506.0|            1|
|    Rachael|Rodriguez|         Media buyer|1966-12-02|griffinmary@examp...| +1-791-344-7586x548|544732.0|            1|
|Christopher| Callahan| Exhibition designer|1966-10-23| qwalter@example.com|001-947-745-3939x...|251057.0|            1|
|    Lindsey|   Huerta|Embryologist, cli...|1964-10-20|  psmith@example.net|   527.934.6665x1378|878257.0|            1|
|      David|   Harris|   Company secretary|1990-04-13|     nli@example.com|001-959-766-1180x...|249553.0|            1|
|      Brian|Hernandez|     Thea

In [20]:
# Create a new column dob_year and register as temp view

emp_temp = spark.sql("""
    select e.*, date_format(dob, 'yyyy') as dob_year from emp_view e
""")


In [21]:
emp_temp.createOrReplaceTempView("emp_temp_view")

In [22]:
spark.sql("select * from emp_temp_view").show()

+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+--------+
|first_name| last_name|           job_title|       dob|               email|               phone|  salary|department_id|dob_year|
+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+--------+
|   Richard|  Morrison|Public relations ...|1973-05-05|melissagarcia@exa...|       (699)525-4827|512653.0|            8|    1973|
|     Bobby|  Mccarthy|   Barrister's clerk|1974-04-25|   llara@example.net|  (750)846-1602x7458|999836.0|            7|    1974|
|    Dennis|    Norman|Land/geomatics su...|1990-06-24| jturner@example.net|    873.820.0518x825|131900.0|           10|    1990|
|      John|    Monroe|        Retail buyer|1968-06-16|  erik33@example.net|    820-813-0557x624|485506.0|            1|    1968|
|  Michelle|   Elliott|      Air cabin crew|1975-03-31|tiffanyjohnston@e...|       (705)90

In [23]:
# Join emp and dept - HINTs
# The first query includes a hint /*+ BROADCAST(d) */,
# which suggests to Spark that the dept_view table is small
# and should be broadcast to all worker nodes for faster join execution.
# The second query is the same join operation without the hint,
# allowing Spark to decide the join strategy.



# emp_final = spark.sql("""
#     select /*+ BROADCAST(d) */
#     e.* , d.department_name
#     from emp_view e left outer join dept_view d
#     on e.department_id = d.department_id
# """)


emp_final = spark.sql("""
    select
    e.* , d.department_name
    from emp_view e left outer join dept_view d
    on e.department_id = d.department_id
""")

In [24]:
# Show emp data

emp_final.show()

+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+--------------------+
|first_name| last_name|           job_title|       dob|               email|               phone|  salary|department_id|     department_name|
+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+--------------------+
|   Richard|  Morrison|Public relations ...|1973-05-05|melissagarcia@exa...|       (699)525-4827|512653.0|            8|          Parker PLC|
|     Bobby|  Mccarthy|   Barrister's clerk|1974-04-25|   llara@example.net|  (750)846-1602x7458|999836.0|            7|         Ward-Gordon|
|    Dennis|    Norman|Land/geomatics su...|1990-06-24| jturner@example.net|    873.820.0518x825|131900.0|           10|      Delgado-Keller|
|      John|    Monroe|        Retail buyer|1968-06-16|  erik33@example.net|    820-813-0557x624|485506.0|            1|         Bryan-James|
|  Mic

In [25]:
# Write the data as Table

emp_final.write.format("parquet").saveAsTable("emp_final")

In [26]:
# Read the data from Table

emp_new = spark.sql("select * from emp_final")

In [27]:
emp_new.show()

+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+--------------------+
|first_name| last_name|           job_title|       dob|               email|               phone|  salary|department_id|     department_name|
+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+--------------------+
|   Richard|  Morrison|Public relations ...|1973-05-05|melissagarcia@exa...|       (699)525-4827|512653.0|            8|          Parker PLC|
|     Bobby|  Mccarthy|   Barrister's clerk|1974-04-25|   llara@example.net|  (750)846-1602x7458|999836.0|            7|         Ward-Gordon|
|    Dennis|    Norman|Land/geomatics su...|1990-06-24| jturner@example.net|    873.820.0518x825|131900.0|           10|      Delgado-Keller|
|      John|    Monroe|        Retail buyer|1968-06-16|  erik33@example.net|    820-813-0557x624|485506.0|            1|         Bryan-James|
|  Mic

In [28]:
# Persist metadata



In [29]:
# Show details of metadata

spark.sql("describe extended emp_final").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|          first_name|              string|   NULL|
|           last_name|              string|   NULL|
|           job_title|              string|   NULL|
|                 dob|              string|   NULL|
|               email|              string|   NULL|
|               phone|              string|   NULL|
|              salary|              double|   NULL|
|       department_id|                 int|   NULL|
|     department_name|              string|   NULL|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|             Catalog|       spark_catalog|       |
|            Database|             default|       |
|               Table|           emp_final|       |
|               Owner|                root|       |
|        Created Time|Thu Aug 14 20:21:...|       |
|         La